In [1]:
!pip install seaborn

In [2]:
import os
import gc
import cudf
import pickle
import dask_cudf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm

pd.options.display.max_rows = None
pd.options.display.max_columns = None

%matplotlib inline

In [2]:
%%time
with open('../input/train_data.pkl', 'rb') as f:
    df = pickle.load(f)

CPU times: user 227 ms, sys: 2 s, total: 2.23 s
Wall time: 2.3 s


In [3]:
df.head()

customer_ID         S_2       P_2  \
0  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-03-09  0.938469   
1  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-04-07  0.936665   
2  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-05-28  0.954180   
3  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-06-13  0.960384   
4  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-07-16  0.947248   

       D_39       B_1       B_2       R_1       S_3      D_41       B_3  D_42  \
0  0.001733  0.008724  1.006838  0.009228  0.124035  0.008771  0.004709   NaN   
1  0.005775  0.004923  1.000653  0.006151  0.126750  0.000798  0.002714   NaN   
2  0.091505  0.021655  1.009672  0.006815  0.123977  0.007598  0.009423   NaN   
3  0.002455  0.013683  1.002700  0.001373  0.117169  0.000685  0.005531   NaN   
4  0.002483  0.015193  1.000727  0.007605  0.117325  0.004653  0.009312   NaN   

   D_43      D_44       B_4      D_45       B_5       R_2      D_46      D_47  \
0   NaN  0.000630  0.080986  0.708906  0.170600  0.006204  0.358587  0.525351   
1   NaN  0.002526  0.069419  0.712795  0.113239  0.006206  0.353630  0.521311   
2   NaN  0.007605  0.068839  0.720884  0.060492  0.003259  0.334650  0.524568   
3   NaN  0.006406  0.055630  0.723997  0.166782  0.009918  0.323271  0.530929   
4   NaN  0.007731  0.038862  0.720619  0.143630  0.006667  0.231009  0.529305   

       D_48  D_49       B_6       B_7       B_8      D_50      D_51       B_9  \
0  0.255736   NaN  0.063902  0.059416  0.006466  0.148698  1.335856  0.008207   
1  0.223329   NaN  0.065261  0.057744  0.001614  0.149723  1.339794  0.008373   
2  0.189424   NaN  0.066982  0.056647  0.005126  0.151955  1.337179  0.009355   
3  0.135586   NaN  0.083720  0.049253  0.001418  0.151219  1.339909  0.006782   
4       NaN   NaN  0.075900  0.048918  0.001199  0.154026  1.341735  0.000519   

        R_3      D_52       P_3      B_10  D_53       S_5      B_11       S_6  \
0  0.001423  0.207334  0.736463  0.096219   NaN  0.023381  0.002768  0.008322   
1  0.001984  0.202778  0.720886  0.099804   NaN  0.030599  0.002749  0.002482   
2  0.007426  0.206629  0.738044  0.134073   NaN  0.048367  0.010077  0.000530   
3  0.003515  0.208214  0.741813  0.134437   NaN  0.030063  0.009667  0.000783   
4  0.001362  0.205468  0.691986  0.121518   NaN  0.054221  0.009484  0.006698   

       D_54       R_4       S_7      B_12       S_8      D_55      D_56  \
0  1.001519  0.008298  0.161345  0.148266  0.922998  0.354596  0.152025   
1  1.009033  0.005136  0.140951  0.143530  0.919414  0.326757  0.156201   
2  1.009184  0.006961  0.112229  0.137014  1.001977  0.304124  0.153795   
3  1.007455  0.008706  0.102838  0.129017  0.704016  0.275055  0.155772   
4  1.003738  0.003846  0.094311  0.129539  0.917133  0.231110  0.154914   

       B_13       R_5      D_58       S_9      B_14      D_59      D_60  \
0  0.118075  0.001882  0.158612  0.065728  0.018385  0.063646  0.199617   
1  0.118737  0.001610  0.148459  0.093935  0.013035  0.065501  0.151387   
2  0.114534  0.006328  0.139504  0.084757  0.056653  0.070607  0.305883   
3  0.120740  0.004980  0.138100  0.048382  0.012498  0.065926  0.273553   
4  0.095178  0.001653  0.126443  0.039259  0.027897  0.063697  0.233103   

       D_61      B_15      S_11      D_62 D_63 D_64      D_65      B_16  B_17  \
0  0.308233  0.016361  0.401619  0.091071   CR    O  0.007126  0.007665   NaN   
1  0.265026  0.017688  0.406326  0.086805   CR    O  0.002413  0.007148   NaN   
2  0.212165  0.063955  0.406768  0.094001   CR    O  0.001878  0.003636   NaN   
3  0.204300  0.022732  0.405175  0.094854   CR    O  0.005899  0.005896   NaN   
4  0.175655  0.031171  0.487460  0.093915   CR    O  0.009479  0.001714   NaN   

       B_18      B_19  D_66      B_20  D_68      S_12       R_6      S_13  \
0  0.652984  0.008520   NaN  0.004730   6.0  0.272008  0.008363  0.515222   
1  0.647093  0.002238   NaN  0.003879   6.0  0.188970  0.004030  0.509048   
2  0

In [4]:
df.shape

(5531451, 190)

## customer_ID

In [5]:
# Check duplicated ID with test data
test = pd.read_csv('../input/test_data.csv', chunksize=800000, usecols=['customer_ID'])
for t in test:
    test = t
    break

# No duplicated ID
display(df[df['customer_ID'].isin(test['customer_ID'].unique())])

Empty DataFrame
Columns: [customer_ID, S_2, P_2, D_39, B_1, B_2, R_1, S_3, D_41, B_3, D_42, D_43, D_44, B_4, D_45, B_5, R_2, D_46, D_47, D_48, D_49, B_6, B_7, B_8, D_50, D_51, B_9, R_3, D_52, P_3, B_10, D_53, S_5, B_11, S_6, D_54, R_4, S_7, B_12, S_8, D_55, D_56, B_13, R_5, D_58, S_9, B_14, D_59, D_60, D_61, B_15, S_11, D_62, D_63, D_64, D_65, B_16, B_17, B_18, B_19, D_66, B_20, D_68, S_12, R_6, S_13, B_21, D_69, B_22, D_70, D_71, D_72, S_15, B_23, D_73, P_4, D_74, D_75, D_76, B_24, R_7, D_77, B_25, B_26, D_78, D_79, R_8, R_9, S_16, D_80, R_10, R_11, B_27, D_81, D_82, S_17, R_12, B_28, R_13, D_83, ...]
Index: []

In [6]:
# unique customer_ID: 458913 < train data records: 5531451
# Train data have duplicated customer_ID
df['customer_ID'].nunique()

458913

In [7]:
# Test data are same
test['customer_ID'].nunique()

65101

In [8]:
# Extract one customer_ID
# It looks like transaction record
# S_2 looks like transaction date
_id = df['customer_ID'].unique()[0]
df[df['customer_ID'] == _id]

customer_ID         S_2       P_2  \
0   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-03-09  0.938469   
1   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-04-07  0.936665   
2   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-05-28  0.954180   
3   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-06-13  0.960384   
4   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-07-16  0.947248   
5   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-08-04  0.945964   
6   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-09-18  0.940705   
7   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-10-08  0.914767   
8   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-11-20  0.950845   
9   0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-12-04  0.868580   
10  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2018-01-11  0.909811   
11  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2018-02-21  0.937349   
12  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2018-03-13  0.934745   

        D_39       B_1       B_2       R_1       S_3      D_41       B_3  \
0   0.001733  0.008724  1.006838  0.009228  0.124035  0.008771  0.004709   
1   0.005775  0.004923  1.000653  0.006151  0.126750  0.000798  0.002714   
2   0.091505  0.021655  1.009672  0.006815  0.123977  0.007598  0.009423   
3   0.002455  0.013683  1.002700  0.001373  0.117169  0.000685  0.005531   
4   0.002483  0.015193  1.000727  0.007605  0.117325  0.004653  0.009312   
5   0.001746  0.007863  1.005006  0.004220  0.110946  0.009857  0.009866   
6   0.002183  0.018859  1.008024  0.004509  0.103329  0.006603  0.000783   
7   0.003029  0.014324  1.000242  0.000263  0.108115  0.009527  0.007836   
8   0.009896  0.016888  1.003995  0.001789  0.102792  0.002519  0.009817   
9   0.001082  0.001930  1.007504  0.001772  0.100470  0.004626  0.006073   
10  0.005715  0.002829  1.004798  0.008175  0.098882  0.001853  0.003238   
11  0.002425  0.019837  1.008307  0.000607  0.102985  0.006174  0.007454   
12  0.009119  0.009382  1.007647  0.006104  0.135021  0.001604  0.007174   

    D_42  D_43      D_44       B_4      D_45       B_5       R_2      D_46  \
0    NaN   NaN  0.000630  0.080986  0.708906  0.170600  0.006204  0.358587   
1    NaN   NaN  0.002526  0.069419  0.712795  0.113239  0.006206  0.353630   
2    NaN   NaN  0.007605  0.068839  0.720884  0.060492  0.003259  0.334650   
3    NaN   NaN  0.006406  0.055630  0.723997  0.166782  0.009918  0.323271   
4    NaN   NaN  0.007731  0.038862  0.720619  0.143630  0.006667  0.231009   
5    NaN   NaN  0.004128  0.027265  0.721371  0.138303  0.006437  0.275963   
6    NaN   NaN  0.009397  0.006608  0.721725  0.108522  0.002254  0.307869   
7    NaN   NaN  0.001399  0.005677  0.723293  0.187638  0.008154  0.519619   
8    NaN   NaN  0.002578  0.013713  0.730918  0.178574  0.009845  0.430077   
9    NaN   NaN  0.007595  0.005182  0.735001  0.189011  0.001795  0.430871   
10   NaN   NaN  0.002389  0.000836  0.730121  0.085317  0.004313  0.503054   
11   NaN   NaN  0.005109  0.080204  0.740062  0.132626  0.001436  0.425848   
12   NaN   NaN  0.003258  0.070793  0.740102  0.231717  0.008309  0.420521   

        D_47      D_48  D_49       B_6       B_7       B_8      D_50  \
0   0.525351  0.255736   NaN  0.063902  0.059416  0.006466  0.148698   
1   0.521311  0.223329   NaN  0.065261  0.057744  0.001614  0.149723   
2   0.524568  0.189424   NaN  0.066982  0.056647  0.005126  0.151955   
3   0.530929  0.135586   NaN  0.083720  0.049253  0.001418  0.151219   
4   0.529305       NaN   NaN  0.075900  0.048918  0.001199  0.154026   
5   0.529762       NaN   NaN  0.095784  0.035738  0.004577  0.149506   
6   0.535747       NaN   NaN  0.104519  0.014672  0.006424  0.146442   
7   0.533295  0.257569   NaN  0.108926  0.005989  0.000839  0.147615   
8   0.539065  0.189361   NaN  0.122928  0.017307  0.004835  0.152756   
9   0.542119  0.403448   NaN  0.221899  0.001681  0.00209

In [9]:
del test, df
gc.collect()

0

## Data Stats

- D_* = Delinquency variables
- S_* = Spend variables
- P_* = Payment variables
- B_* = Balance variables
- R_* = Risk variables

### D_*

Delinquency variablesDelinquency variables

- categorical features
```
'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'
```

### Load Dataset

In [3]:
with open('../input/train_data_D.pkl', 'rb') as f:
    df = pickle.load(f)
    
features = [f for f in df.columns if f.startswith('D')]

In [4]:
df.head()

customer_ID      D_39      D_41  \
0  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001733  0.008771   
1  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.005775  0.000798   
2  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.091505  0.007598   
3  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.002455  0.000685   
4  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.002483  0.004653   

   D_42  D_43      D_44      D_45      D_46      D_47      D_48  D_49  \
0   NaN   NaN  0.000630  0.708906  0.358587  0.525351  0.255736   NaN   
1   NaN   NaN  0.002526  0.712795  0.353630  0.521311  0.223329   NaN   
2   NaN   NaN  0.007605  0.720884  0.334650  0.524568  0.189424   NaN   
3   NaN   NaN  0.006406  0.723997  0.323271  0.530929  0.135586   NaN   
4   NaN   NaN  0.007731  0.720619  0.231009  0.529305       NaN   NaN   

       D_50      D_51      D_52  D_53      D_54      D_55      D_56      D_58  \
0  0.148698  1.335856  0.207334   NaN  1.001519  0.354596  0.152025  0.158612   
1  0.149723  1.339794  0.202778   NaN  1.009033  0.326757  0.156201  0.148459   
2  0.151955  1.337179  0.206629   NaN  1.009184  0.304124  0.153795  0.139504   
3  0.151219  1.339909  0.208214   NaN  1.007456  0.275055  0.155772  0.138100   
4  0.154026  1.341735  0.205468   NaN  1.003738  0.231110  0.154914  0.126443   

       D_59      D_60      D_61      D_62 D_63 D_64      D_65  D_66  D_68  \
0  0.063646  0.199617  0.308233  0.091071   CR    O  0.007126   NaN   6.0   
1  0.065501  0.151387  0.265026  0.086805   CR    O  0.002413   NaN   6.0   
2  0.070607  0.305883  0.212165  0.094001   CR    O  0.001878   NaN   6.0   
3  0.065926  0.273553  0.204300  0.094854   CR    O  0.005899   NaN   6.0   
4  0.063697  0.233103  0.175655  0.093915   CR    O  0.009479   NaN   6.0   

       D_69      D_70      D_71      D_72  D_73      D_74      D_75  D_76  \
0  0.009013  0.008342  0.119403  0.004802   NaN  0.080422  0.069067   NaN   
1  0.007842  0.006524  0.140611  0.000094   NaN  0.081413  0.074166   NaN   
2  0.006025  0.002615  0.075868  0.007152   NaN  0.078891  0.076510   NaN   
3  0.005271  0.002052  0.150209  0.005364   NaN  0.077490  0.071547   NaN   
4  0.000152  0.001419  0.096441  0.007972   NaN  0.076561  0.074432   NaN   

   D_77      D_78      D_79      D_80      D_81      D_82      D_83      D_84  \
0   NaN  0.001576  0.004239  0.004061  0.003532  0.506612  0.007043  0.000830   
1   NaN  0.009896  0.007597  0.000127  0.007773  0.500855  0.007789  0.009469   
2   NaN  0.009629  0.003094  0.000954  0.008811  0.504606  0.004093  0.002325   
3   NaN  0.008568  0.003895  0.005665  0.004652  0.508998  0.008817  0.005924   
4   NaN  0.003289  0.002608  0.004465  0.001141  0.506213  0.001845  0.005516   

       D_86  D_87  D_88      D_89      D_91      D_92      D_93      D_94  \
0  0.007084   NaN   NaN  0.002665  1.503673  1.006133  0.003569  0.008871   
1  0.006677   NaN   NaN  0.002508  1.503577  1.005791  0.000571  0.000391   
2  0.001185   NaN   NaN  0.009634  1.503359  1.005801  0.007425  0.009234   
3  0.003324   NaN   NaN  0.007791  1.503701  1.007036  0.000664  0.003200   
4  0.001504   NaN   NaN  0.005158  1.509905  1.002915  0.003079  0.003845   

       D_96     D_102     D_103     D_104     D_105  D_106     D_107  D_108  \
0  0.004950  0.766688  1.008691  1.004587  0.893734    NaN  0.670041    NaN   
1  0.003180  0.786007  1.000084  1.004118  0.906841    NaN  0.668647    NaN   
2  0.005433  0.806840  1.003014  1.009285  0.928719    NaN  0.670901    NaN   
3  0.000063  0.808214  1.001517  1.004514  0.935383    NaN  0.672620    NaN   
4  0.000535  0.822281  1.006125  1.005735  0.953363    NaN  0.673869    NaN   

      D_109  D_110  D_111     D_112     D_113  D_114     D_115  D_116  D_117  \
0  0.004326    NaN    NaN  1.007336  0.007871    1.0  0.238250    0.0    4.0   
1  0.008707    NaN    NaN  1.007653  0.003444    1.0  0.247217    0.0    4.0   
2  0.004092    NaN    NaN  1.004312  0.003269    1.0  0.239867    0

### Null Rate

In [5]:
# Nulls(%)
null_rate = pd.DataFrame(df.isnull().sum() / len(df) * 100, columns=['null']).sort_values(by='null', ascending=False).T

In [6]:
null_rate

D_87       D_88      D_108     D_110     D_111       D_73  \
null  99.930127  99.891457  99.476846  99.43353  99.43353  98.990211   

          D_134      D_135      D_136      D_138      D_137      D_106  \
null  96.480146  96.480146  96.480146  96.480146  96.480146  90.213255   

          D_132       D_49       D_76       D_66       D_42      D_142  \
null  90.191055  90.137597  88.746226  88.730733  85.694278  82.926577   

           D_53       D_82       D_50      D_105       D_56       D_77  \
null  73.842921  73.373406  56.809723  54.622756  54.071581  45.447605   

           D_43       D_46       D_62       D_48       D_61      D_44  \
null  29.981211  21.905627  13.706367  12.993426  10.811847  4.959259   

          D_78      D_64      D_68      D_83      D_69     D_55     D_115  \
null  4.959259  3.931012  3.914036  3.515768  3.515768  3.34095  3.194749   

         D_120     D_113     D_121     D_122     D_123     D_118     D_116  \
null  3.194749  3.194749  3.194749  3.194749  3.194749  3.194749  3.194749   

         D_124     D_125     D_114     D_117     D_119     D_91     D_126  \
null  3.194749  3.194749  3.194749  3.194749  3.194749  2.84222  2.111851   

          D_59     D_128     D_129     D_130     D_139     D_141     D_143  \
null  1.929421  1.835829  1.835829  1.835829  1.835829  1.835829  1.835829   

         D_131     D_145     D_104     D_107     D_103      D_70      D_79  \
null  1.835829  1.835829  1.835829  1.835829  1.835829  1.715951  1.372859   

         D_133     D_144     D_102     D_140      D_89      D_84      D_52  \
null  0.772239  0.736281  0.734979  0.734563  0.534453  0.534453  0.534453   

          D_81      D_72      D_80      D_74     D_112      D_45      D_54  \
null  0.464381  0.428604  0.393622  0.393622  0.047908  0.036464  0.036446   

          D_41     D_109  D_60  D_47  D_58  D_51  D_127  D_63  D_65  D_71  \
null  0.036446  0.028871   0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   

      D_75  D_86  D_39  D_92  D_93  D_94  D_96  customer_ID  
null   0.0   0.0   0.0   0.0   0.0   0.0   0.0          0.0

It looks same null rate(3.194749%)
```
D_115, D_120, D_113, D_123, D_118, D_116, D_124, D_125, D_114, D_117, D_119
```

It looks same null rate(1.835829%)
```
D_128, D_129, D_130, D_139, D_141, D_143, D_131, D_145, D_104, D_107, D_103
```


It seems that these features are related each other.

In [ ]:
ncols = 6
nrows = int(np.ceil(len(features) / ncols))
df_sampled = df.sample(frac=0.01, random_state=0)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(14, int(2 * nrows)))

for ax, f in zip(axes.ravel(), features):
    sns.histplot(df_sampled[f], ax=ax)
    ax.set_title(f)
    
plt.tight_layout()
plt.show()

### S_*

Spend variables

- categorical features
```
None
```

In [ ]:
with open('../input/train_data_S.pkl', 'rb') as f:
    df = pickle.load(f)
    
features = [f for f in df.columns if f.startswith('D')]

In [ ]:
df.head()

### Null Rate